### Tpot

In [1]:
import pandas as pd
import numpy as np
import datetime
import sklearn
from tpot import TPOTRegressor
from sklearn.cross_validation import train_test_split
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

C:\Users\achar\Anaconda3\lib\site-packages\deap\tools\_hypervolume\pyhv.py:33: ImportWarning: Falling back to the python version of hypervolume module. Expect this to be very slow.
  "module. Expect this to be very slow.", ImportWarning)
C:\Users\achar\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\achar\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)
C:\Users\achar\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory C:\Users\achar\Anaconda3\lib\site-packages\mpl_toolkits: missing __init__
  _warnings.warn(msg.format(portions[0]), ImportWarning)
C:\Users\achar\Anaconda3\lib\importlib\_bootstrap_external.py:426: ImportWarning: Not importing directory c:\users\achar\anaconda3\lib\site-package

In [2]:
data = pd.read_csv('energydata_Transformation.csv', parse_dates=['date'])

In [3]:
weekType = pd.get_dummies(data['weekType'], prefix = 'weekType')
day_of_week = pd.get_dummies(data['day_of_week'], prefix = 'day_of_week')
#['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
#['Weekend', 'Weekday']

# Concat above dummies variable dataframe to the main dataframe
data = pd.concat((data,weekType),axis=1)
data = pd.concat((data,day_of_week),axis=1)

# Drop the WeekStatus and Day_of_week column
data = data.drop(['weekType','day_of_week','date', 'time'],axis=1)

In [4]:
check_outliers = ['Appliances', 'temp_kitchen', 'hum_kitchen', 'temp_living',
       'hum_living', 'temp_laundry', 'hum_laundry', 'temp_office',
       'hum_office', 'temp_bathroom', 'hum_bathroom', 'temp_building_out',
       'hum_building_out', 'temp_ironing', 'hum_ironing', 'temp_teenRoom',
       'hum_teenRoom', 'hum_parentRoom', 'Pressure', 'hum_out', 'Windspeed',
       'Visibility', 'Tdewpoint', 'rv1']

for x in data[check_outliers]:
    data = data[np.abs(data[x]-data[x].mean()) <= (3*data[x].std())]

In [5]:
data_train,data_test = train_test_split(data,train_size=0.7,random_state=42)
x_train=data_train.iloc[:,1:]
y_train=data_train['Appliances']
scaler.fit(x_train)
x_train_sc=scaler.transform(x_train)
x_test=data_test.iloc[:,1:]
y_test=data_test['Appliances']
x_test_sc=scaler.transform(x_test)

In [6]:
my_tpot = TPOTRegressor(generations=5, population_size = 20 , verbosity= 2)  
my_tpot.fit(x_train_sc, y_train)

C:\Users\achar\Anaconda3\lib\importlib\_bootstrap.py:219: ImportWarning: can't resolve package from __spec__ or __package__, falling back on __name__ and __path__
  return f(*args, **kwds)


Generation 1 - Current best internal CV score: -2351.5466157697742
Generation 2 - Current best internal CV score: -2351.5466157697742
Generation 3 - Current best internal CV score: -2351.5466157697742
Generation 4 - Current best internal CV score: -2351.5466157697742
Generation 5 - Current best internal CV score: -2301.0843441249162

Best pipeline: ExtraTreesRegressor(PolynomialFeatures(input_matrix, degree=2, include_bias=False, interaction_only=False), bootstrap=False, max_features=0.15000000000000002, min_samples_leaf=5, min_samples_split=6, n_estimators=100)


TPOTRegressor(config_dict=None, crossover_rate=0.1, cv=5,
       disable_update_check=False, early_stop=None, generations=5,
       max_eval_time_mins=5, max_time_mins=None, memory=None,
       mutation_rate=0.9, n_jobs=1, offspring_size=None,
       periodic_checkpoint_folder=None, population_size=20,
       random_state=None, scoring=None, subsample=1.0, use_dask=False,
       verbosity=2, warm_start=False)

In [7]:
print(my_tpot.score(x_train_sc, y_train))

-900.2838463288963


In [9]:
print(my_tpot.score(x_test_sc, y_test))

-2394.4671160521625
